# Q1.
Write a python function program to demonstrate the working of the decision tree based C4.5 algorithms
without using scikit-learn library. Use following data set for building the decision tree and apply this
knowledge to classify a new sample.
The dataset has three attributes: Outlook (Sunny, Overcast, Rainy), Temperature, Humidity and Wind (Weak,
Strong). The target attribute is Play Tennis (Yes/No).

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [25]:
outlook = ['sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain', 'overcast', 'sunny', 'sunny', 'rain', 'sunny', 'overcast', 'overcast', 'rain']
temperature = [85, 80, 83, 70, 68, 65, 64, 72, 69, 75, 75, 72, 81, 71]
humidity = [85, 90, 78, 96, 80, 70, 65, 95, 70, 80, 70, 90, 75, 80]
wind = ['weak', 'strong', 'weak', 'weak','weak', 'strong', 'strong', 'weak', 'weak', 'weak', 'strong', 'strong', 'weak', 'strong']
decision = ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']

data = {'Outlook': outlook, 'Temp': temperature, 'Humidity': humidity, 'Wind': wind, 'Decision': decision}

df = pd.DataFrame(data)

def convertTemp(num):
    if int(num)>=80:
        return "Hot"
    elif int(num)<70:
        return "Cool"
    else:
        return "Mild"

def convertHumidity(num):
    if int(num)>80:
        return "High"
    else:
        return "Normal"

df['Temp'] = df['Temp'].astype(str)
df['Humidity'] = df['Humidity'].astype(str)

df['Temp'] = df['Temp'].apply(convertTemp)
df['Humidity'] = df['Humidity'].apply(convertHumidity)

df

,Outlook,Temp,Humidity,Wind,Decision
0,sunny,Hot,High,weak,no
1,sunny,Hot,High,strong,no
2,overcast,Hot,Normal,weak,yes
3,rain,Mild,High,weak,yes
4,rain,Cool,Normal,weak,yes
5,rain,Cool,Normal,strong,no
6,overcast,Cool,Normal,strong,yes
7,sunny,Mild,High,weak,no
8,sunny,Cool,Normal,weak,yes
9,rain,Mild,Normal,weak,yes


In [36]:
def entropy(data):
    p,n = 0,0
    x = data.value_counts("Decision")
    if len(x) > 1:
        if data.iat[0,-1] == "no":
            n,p = x[0],x[1]
        else:
            p,n = x[0],x[1]
    else:
        if data.iat[0,-1] == "no":
            n = x[0]
        else:
            p = x[0]
    p_ratio = p/(p+n)
    n_ratio = 1 - p_ratio
    entropy_p = -p_ratio*math.log2(p_ratio) if p_ratio != 0 else 0
    entropy_n = - n_ratio*math.log2(n_ratio) if n_ratio !=0 else 0
    return entropy_p + entropy_n

def gain_ratio(data,feature):
    unique_values = data[feature].unique()
    feature_weights = [len(data[data[feature] == val]) for val in unique_values]
    entropies = [entropy(data[data[feature] == val]) for val in unique_values]
    info_gain_feature = sum(feature_weights[i] / sum(feature_weights) * entropies[i] for i in range(len(unique_values)))
    info_gain = entropy(data) - info_gain_feature
    split_info = sum(-1*feature_weights[i] / sum(feature_weights) * math.log2(feature_weights[i] / sum(feature_weights)) for i in range(len(unique_values)))
    gain_ratio = info_gain/split_info
    return gain_ratio



entropy(df)

print(gain_ratio(df,"Outlook"))
print(gain_ratio(df,"Temp"))
print(gain_ratio(df,"Humidity"))
print(gain_ratio(df,"Wind"))

0.15642756242117528
0.018772646222418813
0.1087366695918781
0.048848615511520824


/tmp/ipykernel_5742/4121606587.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  n,p = x[0],x[1]
/tmp/ipykernel_5742/4121606587.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  n,p = x[0],x[1]
/tmp/ipykernel_5742/4121606587.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_5742/4121606587.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer key

In [35]:
class Node:
    def __init__(self, data, parent_feature=None, parent_feature_value=None):
        self.data = data  
        self.parent_feature = parent_feature  
        self.parent_feature_value = parent_feature_value  
        self.children = {} 

def c45_decision_tree(data, features, target, parent_feature=None, parent_feature_value=None):
    node = Node(data, parent_feature, parent_feature_value)    
    if len(data['Decision'].unique()) == 1:
        return data['Decision'].values[0]    
    if len(features) == 0:
        return data['Decision'].mode().values[0]    
    gain_ratios = {feature: gain_ratio(data, feature) for feature in features}
    best_feature = max(gain_ratios, key=gain_ratios.get)    
    for value in data[best_feature].unique():
        child_data = data[data[best_feature] == value].drop([best_feature], axis=1)
        node.children[value] = c45_decision_tree(child_data, features.drop(best_feature), target, best_feature, value)
    return node

features = df.columns[:-1]
target = 'Decision'
decision_tree = c45_decision_tree(df, features, target)

def print_c45_decision_tree(node, indent=""):
    if isinstance(node, Node):
        if node.parent_feature is not None:
            print(indent + "Feature: " + node.parent_feature)
            print(indent + "Parent Feature Value: " + str(node.parent_feature_value))
        
        if len(node.children) == 0:
            print(indent + "Decision: " + str(node.data['Decision'].values[0]))
        else:
            for value, child_node in node.children.items():
                print(indent + "Value: " + str(value))
                print_decision_tree(child_node, indent + "  ")
    else:
        print(indent + "Decision: " + str(node))

print_c45_decision_tree(decision_tree)

Value: sunny
  Feature: Outlook
  Parent Feature Value: sunny
  Value: High
    Decision: no
  Value: Normal
    Decision: yes
Value: overcast
  Decision: yes
Value: rain
  Feature: Outlook
  Parent Feature Value: rain
  Value: weak
    Decision: yes
  Value: strong
    Decision: no


/tmp/ipykernel_5742/2750526191.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/2750526191.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_5742/2750526191.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/2750526191.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer key

# Q2.
Write a python function program to demonstrate the working of the decision tree based CART algorithms
without using scikit-learn library. Use Q. No. 1 data set for building the decision tree and apply this knowledge
to classify a new sample.

In [39]:
def gini(data):
    p,n = 0,0
    x = data.value_counts("Decision")
    if len(x) > 1:
        if data.iat[0,-1] == "no":
            n,p = x[0],x[1]
        else:
            p,n = x[0],x[1]
    else:
        if data.iat[0,-1] == "no":
            n = x[0]
        else:
            p = x[0]
    p_ratio = p/(p+n)
    n_ratio = 1 - p_ratio
    gini_p = (p_ratio)**2
    gini_n = (n_ratio)**2
    return 1 - gini_p - gini_n

def gini_feature(data,feature):
    unique_values = data[feature].unique()
    feature_weights = [len(data[data[feature] == val]) for val in unique_values]
    ginies = [gini(data[data[feature] == val]) for val in unique_values]
    gini_feature_val = sum(feature_weights[i] / sum(feature_weights) * ginies[i] for i in range(len(unique_values)))
    return gini_feature_val

entropy(df)
print(gini_feature(df,"Outlook"))
print(gini_feature(df,"Temp"))
print(gini_feature(df,"Humidity"))
print(gini_feature(df,"Wind"))

0.34285714285714286
0.44047619047619047
0.3936507936507937
0.42857142857142855


/tmp/ipykernel_5742/4121606587.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  n,p = x[0],x[1]
/tmp/ipykernel_5742/4078221434.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  n,p = x[0],x[1]
/tmp/ipykernel_5742/4078221434.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_5742/4078221434.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer key

In [40]:
class Node:
    def __init__(self, data, parent_feature=None, parent_feature_value=None):
        self.data = data  
        self.parent_feature = parent_feature  
        self.parent_feature_value = parent_feature_value  
        self.children = {}  

def build_cart_decision_tree(data, features, target, parent_feature=None, parent_feature_value=None):
    node = Node(data, parent_feature, parent_feature_value)
    if len(data['Decision'].unique()) == 1:
        return data['Decision'].values[0]
    if len(features) == 0:
        return data['Decision'].mode().values[0]
    gini_indices = {feature: gini_feature(data, feature) for feature in features}
    best_feature = min(gini_indices, key=gini_indices.get)
    for value in data[best_feature].unique():
        child_data = data[data[best_feature] == value].drop([best_feature], axis=1)
        node.children[value] = build_cart_decision_tree(child_data, features.drop(best_feature), target, best_feature, value)
    return node
features = df.columns[:-1]
target = 'Decision'

decision_tree = build_cart_decision_tree(df, features, target)
def print_cart_decision_tree(node, indent=""):
    if isinstance(node, Node):
        if node.parent_feature is not None:
            print(indent + "Feature: " + node.parent_feature)
            print(indent + "Parent Feature Value: " + str(node.parent_feature_value))

        if len(node.children) == 0:
            print(indent + "Decision: " + str(node.data['Decision'].values[0]))
        else:
            for value, child_node in node.children.items():
                print(indent + "Value: " + str(value))
                print_cart_decision_tree(child_node, indent + "  ")
    else:
        print(indent + "Decision: " + str(node))
        
print_cart_decision_tree(decision_tree)

Value: sunny
  Feature: Outlook
  Parent Feature Value: sunny
  Value: High
    Decision: no
  Value: Normal
    Decision: yes
Value: overcast
  Decision: yes
Value: rain
  Feature: Outlook
  Parent Feature Value: rain
  Value: weak
    Decision: yes
  Value: strong
    Decision: no


/tmp/ipykernel_5742/4078221434.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  n,p = x[0],x[1]
/tmp/ipykernel_5742/4078221434.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_5742/4078221434.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/4078221434.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer key

# Q3.
Write a python function program to demonstrate the working of the decision tree based C4.5 and CART
algorithms without and with using scikit-learn library. Using the following dataset, apply aforementioned
algorithms. The attributes are Income (Low, Medium, High) and Credit (Good, Bad), and the target is Loan
Approved (Yes/No).

In [42]:
income = ['low', 'low', 'medium', 'medium', 'high', 'high']
credit = ['good', 'bad', 'good', 'bad', 'good', 'bad']
approved = ['yes', 'no', 'yes', 'yes', 'yes', 'no']

data = {'Income' : income, 'Credit' : credit, 'Decision' : approved}

df = pd.DataFrame(data)
df

,Income,Credit,Decision
0,low,good,yes
1,low,bad,no
2,medium,good,yes
3,medium,bad,yes
4,high,good,yes
5,high,bad,no


## C4.5

In [47]:
entropy(df)

print(gain_ratio(df,"Income"))
print(gain_ratio(df,"Credit"))

features = df.columns[:-1]
target = 'Decision'
decision_tree = c45_decision_tree(df, features, target)

print_c45_decision_tree(decision_tree)

0.15876032857139014
0.4591479170272448
Value: good
  Decision: yes
Value: bad
  Feature: Credit
  Parent Feature Value: bad
  Value: low
    Decision: no
  Value: medium
    Decision: yes
  Value: high
    Decision: no


/tmp/ipykernel_5742/4121606587.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/4121606587.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/4121606587.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_5742/4121606587.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer key

## CART

In [48]:
entropy(df)

print(gini_feature(df,"Income"))
print(gini_feature(df,"Credit"))

features = df.columns[:-1]
target = 'Decision'
decision_tree = c45_decision_tree(df, features, target)

print_cart_decision_tree(decision_tree)

0.3333333333333333
0.22222222222222215
Value: good
  Decision: yes
Value: bad
  Feature: Credit
  Parent Feature Value: bad
  Value: low
    Decision: no
  Value: medium
    Decision: yes
  Value: high
    Decision: no


/tmp/ipykernel_5742/4121606587.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/4078221434.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_5742/4078221434.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_5742/4078221434.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer key